# Final calibration of PDM

In [1]:
import os
import numpy as np
import pandas as pd
import geopandas as gpd
from pathlib import Path
import matplotlib.pyplot as plt
import hvplot
import hvplot.pandas
import warnings
import pyswarms as ps
from datetime import timedelta, datetime
from dateutil.relativedelta import relativedelta
from sklearn.preprocessing import StandardScaler

pad = Path(os.getcwd())
if pad.name == "model_training_and_calibration":
    pad_correct = pad.parent
    os.chdir(pad_correct)
from functions.PDM import (PDM, parameter_sampling, 
                           Nelder_Mead_calibration,PDM_calibration_wrapper_PSO)
from functions.performance_metrics import NSE, mNSE, FHV, NSE_LF
from functions.plotting_functions import plot_FDC

warnings.filterwarnings(action = 'ignore', category= RuntimeWarning)
warnings.filterwarnings(action = 'ignore', category= UserWarning)

## Read in and process initial data

In [ ]:
parameters_initial = pd.DataFrame({
    'cmax': 944.6,
    'cmin':59.8,
    'b':0.51,
    'be':1.5,
    'k1':14.7,
    'k2':4.2,
    'kb':175.5,
    'kg':10265,
    'St': 25.3,
    'bg':1.00000,
    'tdly': 12.00000,
    'qconst':-0.13,
}, dtype = np.float32, index =[0])
display(parameters_initial)



zwalm_shape = gpd.read_file('data/Zwalm_shape/zwalm_shapefile_david_4326.shp')
area_zwalm_new = np.single(zwalm_shape.OPPERVL[0]/10**6)
print('Area of the Zwalm by shapefile: ' + str(area_zwalm_new) + '[km^2]')

Defining the bounds of the parameters as described in thesis.


In [ ]:
lower_bound = np.array([160,0,0.1,1,0.9,0.1,0,700,0,1,0,-0.3]) 
upper_bound = np.array([5000,300,2,2,40,15,5000,25000,150,1.000000000000001,24,0.03])
bounds_list = []
for i in range(len(lower_bound)):
    bounds_list.append((lower_bound[i],upper_bound[i]))
#bounds_opt = tuple(bounds_list)
bounds_opt = bounds_list
print(bounds_opt)

In [ ]:
bounds_dict = {}
for i in range(len(bounds_opt)):
    bounds_dict[parameters_initial.columns.values[i]] = bounds_opt[i]
bounds_dict

In [5]:
preprocess_output_folder = Path('data/Zwalm_data/preprocess_output')
p_ep_zwalm = pd.read_pickle(preprocess_output_folder/'Final_Forcings_PDM.pkl')


Q_output_folder = Path("data/Zwalm_data/output_Q")
Q_day = pd.read_pickle(Q_output_folder/"Q_data.pkl")

Q_day = Q_day.set_index('Timestamp')



In [ ]:
warmup_months = 12
start_p1 = p_ep_zwalm['Timestamp'].iloc[0]
start_endofwarmup_p1 = start_p1 + relativedelta(months = warmup_months)
end_p1 =  pd.Timestamp(datetime(year = 2017, month = 12, day = 14, hour = 23))

print('Characteristics of period 1: start = '  + str(start_p1) + ', start of post warmup = ' + str(start_endofwarmup_p1) + ' and end = ' + str(end_p1))

start_p2 = pd.Timestamp(datetime(year = 2017, month= 12, day = 15, hour = 0))
start_endofwarmup_p2 = start_p2 + relativedelta(months = warmup_months)
end_p2 = p_ep_zwalm['Timestamp'].iloc[-1]
print('Characteristics of period 2: start = '  + str(start_p2) + ', start of post warmup = ' + str(start_endofwarmup_p2) + ' and end = ' + str(end_p2))

p1_period_excl_warmup = pd.date_range(start_endofwarmup_p1,end_p1,
freq = 'D') #used for scoring the model 
p1_period = pd.date_range(start_p1, end_p1, freq = 'H')
p2_period_excl_warmup = pd.date_range(start_endofwarmup_p2,end_p2,
freq = 'D') #used for scoring the model 
p2_period = pd.date_range(start_p2, end_p2, freq = 'H')
p_all_nowarmup = pd.date_range(start_endofwarmup_p1, end_p2)
p_all = pd.date_range(start_p1, end_p2)

#now subdivide ep data on p1 and p2
#for ease of selecting data, set time as index!
#select forcings for p1 period
p_zwalm_p1 = p_ep_zwalm.set_index('Timestamp').loc[p1_period]
#select forcings for p2 period
p_zwalm_p2 = p_ep_zwalm.set_index('Timestamp').loc[p2_period]


## Check initial performance of the model

In [ ]:
deltat = np.single(1) #internal resolution =  1 hour
deltat_out = np.single(24) #output resolution = 24 hour
pd_zwalm_out_initial = PDM(P = p_ep_zwalm['total_precipitation_sum'].values, 
    EP = p_ep_zwalm['potential_evaporation_sum'].values,
    t = p_ep_zwalm['Timestamp'].values,
    area = area_zwalm_new, deltat = deltat, deltatout = deltat_out ,
    parameters = parameters_initial)
pd_zwalm_out_initial = pd_zwalm_out_initial.set_index(['Time'])
pd_zwalm_out_initial_p2 = pd_zwalm_out_initial[start_p2:end_p2] #no warmup!
pd_zwalm_out_initial_p1 = pd_zwalm_out_initial[start_endofwarmup_p1:end_p1]

kwargs_p1 = {'Qmod':pd_zwalm_out_initial_p1['qmodm3s'].values, 'Qobs':Q_day.loc[start_endofwarmup_p1:end_p1,'streamflow'].values}
kwargs_p2 = {'Qmod':pd_zwalm_out_initial_p2['qmodm3s'].values, 'Qobs':Q_day.loc[start_p2:end_p2,'streamflow'].values}
kwargs_full = {'Qmod':pd_zwalm_out_initial.loc[start_endofwarmup_p1:end_p2,'qmodm3s'].values, 'Qobs':Q_day.loc[start_endofwarmup_p1:end_p2,'streamflow'].values}
# check (m)NSE and FHV

nse_initial_p2 = NSE(**kwargs_p2)
mnse_initial_p2 = mNSE(**kwargs_p2)
FHV_initial_p2 = FHV(**kwargs_p2)
print('NSE on p2 for initial set:' + str(nse_initial_p2))
print('mNSE on p2 for initial set:' + str(mnse_initial_p2))
print('FHV on p2 for initial set: ' + str(FHV_initial_p2) + '%')

nse_initial_p1 = NSE(**kwargs_p1)
mnse_initial_p1 = mNSE(**kwargs_p1)
FHV_initial_p1 = FHV(**kwargs_p1)
print('NSE on p1 for initial set:' + str(nse_initial_p1))
print('mNSE on p1 for initial set:' + str(mnse_initial_p1))
print('FHV on p1 for initial set: ' + str(FHV_initial_p1) + '%')

nse_initial = NSE(**kwargs_full)
mnse_initial = mNSE(**kwargs_full)
FHV_initial = FHV(**kwargs_full)
print('NSE for initial set:' + str(nse_initial))
print('mNSE for initial set:' + str(mnse_initial))
print('FHV for for initial set: ' + str(FHV_initial) + '%')

In [ ]:
pd_zwalm_out_initial

In [ ]:
#fig, ax = plt.subplots(figsize = (10,6))
plot = Q_day['streamflow'].hvplot(alpha = 0.7, label = 'observed') * pd_zwalm_out_initial['qmodm3s'].hvplot(
    alpha = 0.7, frame_width = 900, frame_height = 400,label = 'modelled', line_dash = 'dashed') 
plot.opts(
    xlabel='Time',
    ylabel='Streamflow (m³/s)'
)

In [ ]:
#plt.style.use('seaborn-v0_8-colorblind')
plt.style.use('default')
fig, ax = plt.subplots(figsize = (9,5), constrained_layout = True)
Q_day['streamflow'].plot(alpha = 0.7, label = '$Q_o$',ax = ax)
pd_zwalm_out_initial['qmodm3s'].plot(
    alpha = 0.7, label = '$Q_{m,init}$', linestyle = '--', ax = ax
)
ax.set_ylabel('$Q$ [m$^3$/s]')
ax.set_xlabel('Time')
ax2 = ax.twinx()
p_zwalm_t = p_ep_zwalm.set_index('Timestamp')
ax2.plot(p_zwalm_t.index, p_zwalm_t['total_precipitation_sum'],c = 'grey')
ax2.invert_yaxis()
ax2.set_ylim(3*np.max(p_zwalm_t['total_precipitation_sum']),0)
ax2.set_ylabel('$P$ [mm/h]')
# p_zwalm_t['P_thiessen'].plot(ax = ax2, alpha = 0.1)
# box = ax.get_position()
# ax.set_position([box.x0, box.y0, box.width * 0.8, box.height])

# Put a legend to the right of the current axis
ax.legend(bbox_to_anchor=(1.04, 1), loc="upper left")
pad = Path('Figures/Figures_chapter_PDM')
if not os.path.exists(pad):
    os.makedirs(pad)
fig.savefig(pad/'Q_initial_set.pdf', format = 'pdf')


In [ ]:
fig, ax = plt.subplots()
fig, ax = plot_FDC(Q_day['streamflow'].values, label = '$Q_o$', fig = fig, ax = ax, cutoff_bool=False)
fig, ax = plot_FDC(pd_zwalm_out_initial['qmodm3s'].values, fig = fig, ax = ax, label = '$Q_{m,init}$')

## Nelder-Mead Calibration: starting from multiple initial conditions

In [ ]:
names = parameters_initial.columns.to_list()
n_paramsets = 50
pd_init_params = parameter_sampling(names, bounds_opt, n_paramsets)
pd_init_params.head()
pd_opt_params = parameters_initial.copy()
col_names_perf = ['NSE_cal','NSE_val','NSE_full','mNSE_cal','mNSE_val','mNSE_full']
#empyt dataframe for parameters and performance
append = 1
if append:
    pd_perf = pd.read_csv('data/Zwalm_PDM_parameters/NM_NSE_performances_50.csv')
    pd_opt_params = pd.read_csv('data/Zwalm_PDM_parameters/NM_NSE_parameters_50.csv')
    start = pd_opt_params.shape[0]
else:
    pd_perf = pd.DataFrame(columns = col_names_perf)
    pd_opt_params = pd.DataFrame(columns = names)
    start = 0
display(pd_perf.head())
display(pd_opt_params.head())

In [13]:
#Specifications for calibration 
performance_metric = 'NSE'
P_np = p_zwalm_p1['total_precipitation_sum'].values
EP_np = p_zwalm_p1['potential_evaporation_sum'].values
deltat = np.single(1)
deltatout = np.single(24)
t_model = p1_period.values
t_calibration = p1_period_excl_warmup.values
Qobs = Q_day['streamflow']

In [ ]:
if not os.path.exists('data/Zwalm_PDM_parameters'):
    os.mkdir('data/Zwalm_PDM_parameters')

print(start)

In [ ]:
#set_keepawake(keep_screen_awake=True)
exec_optimisation = True
if exec_optimisation:
        print('test')
        for i in np.arange(start, n_paramsets):
                # Optimize parametersets with NM
                pd_init_temp = pd_init_params.iloc[i,:]
                opt_out_NSE = Nelder_Mead_calibration(
                        pd_init_temp.values, parameters_initial.columns, bounds_opt, performance_metric, P_np, EP_np,
                        area_zwalm_new, deltat, deltatout, t_model, t_calibration, Qobs  
                )
                ## Assign optimized parameter sets
                pd_temp = pd.DataFrame(opt_out_NSE.x.reshape(1,-1), columns = names)
                pd_opt_params = pd.concat([pd_opt_params, pd_temp], axis = 0, ignore_index=True)
                
                ## Quantify performance of parameter sets 
                pd_zwalm_out = PDM(P = p_ep_zwalm['total_precipitation_sum'].values, 
                        EP = p_ep_zwalm['potential_evaporation_sum'].values, t = p_ep_zwalm['Timestamp'].values,
                        area = area_zwalm_new, deltat = deltat, deltatout = deltat_out ,parameters = pd_temp
                )
                pd_zwalm_out = pd_zwalm_out.set_index('Time')
                mnse_cal = mNSE(
                        pd_zwalm_out.loc[start_endofwarmup_p1:end_p1,'qmodm3s'].values,
                        Q_day.loc[start_endofwarmup_p1:end_p1,'streamflow'].values
                )
                nse_cal = NSE(
                        pd_zwalm_out.loc[start_endofwarmup_p1:end_p1,'qmodm3s'].values,
                        Q_day.loc[start_endofwarmup_p1:end_p1,'streamflow'].values
                )
                mnse_val = mNSE(
                        pd_zwalm_out.loc[start_p2:end_p2,'qmodm3s'].values,
                        Q_day.loc[start_p2:end_p2,'streamflow'].values
                )
                nse_val= NSE(
                        pd_zwalm_out.loc[start_p2:end_p2,'qmodm3s'].values,
                        Q_day.loc[start_p2:end_p2,'streamflow'].values
                ) #mistake was made here (mNSE was used) => wrong data in dataframe
                mnse_full = mNSE(
                        pd_zwalm_out.loc[start_endofwarmup_p1:end_p2,'qmodm3s'].values,
                        Q_day.loc[start_endofwarmup_p1:end_p2,'streamflow'].values
                )
                nse_full = NSE(
                        pd_zwalm_out.loc[start_endofwarmup_p1:end_p2,'qmodm3s'].values,
                        Q_day.loc[start_endofwarmup_p1:end_p2,'streamflow'].values
                )
                pd_temp_perf = pd.DataFrame(np.array([nse_cal, nse_val, nse_full, mnse_cal, mnse_val, mnse_full]).reshape(1,-1), columns= col_names_perf, index = [i])
                pd_perf = pd.concat([pd_perf, pd_temp_perf], axis=0, ignore_index = True)

                ## Write out
                pd_opt_params.to_csv('data/Zwalm_PDM_parameters/NM_NSE_parameters_50.csv', mode = 'w', index = False)
                pd_perf.to_csv('data/Zwalm_PDM_parameters/NM_NSE_performances_50.csv', mode = 'w', index = False)
                print('dataset ' + str(i) + ' out of '  + str(n_paramsets) + ' has been calibrated')
#unset_keepawake()

## Assess calibration results

In [ ]:

# File paths
import os
from pathlib import Path
import shutil

# Get current working directory and its parent
pad = Path(os.getcwd())
parent = pad.parent
performance_file = parent / 'data' /'Zwalm_PDM_parameters' / 'NM_NSE_performances_50.csv'
parameters_file = parent / 'data' /'Zwalm_PDM_parameters' / 'NM_NSE_parameters_50.csv'

# Read the performance file and find the row with the highest NSE_cal
performances = pd.read_csv(performance_file)
best_row_index = performances['NSE_val'].idxmax()  # Index of the best NSE_cal
best_performance_row = performances.iloc[best_row_index]
print(best_performance_row)

# Read the parameters file and select the corresponding row
parameters = pd.read_csv(parameters_file)
best_parameters_row = parameters.iloc[best_row_index]

# Format best_par to match the structure of parameters_initial
best_par = pd.DataFrame({
    'cmax': [best_parameters_row['cmax']],
    'cmin': [best_parameters_row['cmin']],
    'b': [best_parameters_row['b']],
    'be': [best_parameters_row['be']],
    'k1': [best_parameters_row['k1']],
    'k2': [best_parameters_row['k2']],
    'kb': [best_parameters_row['kb']],
    'kg': [best_parameters_row['kg']],
    'St': [best_parameters_row['St']],
    'bg': [best_parameters_row['bg']],
    'tdly': [best_parameters_row['tdly']],
    'qconst': [best_parameters_row['qconst']],
}, dtype=np.float32, index=[0])

# Display the result
print("Best Parameters:")
display(best_par)

pd_zwalm_out = PDM(P = p_ep_zwalm['total_precipitation_sum'].values, 
    EP = p_ep_zwalm['potential_evaporation_sum'].values,
    t = p_ep_zwalm['Timestamp'].values,
    area = area_zwalm_new, deltat = deltat, deltatout = deltat_out ,
    parameters = best_par)





pd_zwalm_out.set_index('Time', inplace=True)

# Filter data between 15/12/2017 and 31/12/2022
start_date = '2017-12-15'
end_date = '2022-12-31'

filtered_pd_zwalm_out = pd_zwalm_out.loc[start_date:end_date]
filtered_Q_day = Q_day.loc[start_date:end_date]


plt.figure(figsize=(6.3, 3.5))

# Plot observed streamflow
plt.plot(filtered_Q_day.index, filtered_Q_day['streamflow'], label='Observed Discharge', color='blue', linewidth=1)

# Plot simulated streamflow
plt.plot(filtered_pd_zwalm_out.index, filtered_pd_zwalm_out['qmodm3s'], label='Simulated Discharge', color='red', linewidth = 1)
plotvoor1param = filtered_pd_zwalm_out['qmodm3s']
# Add axis labels and styling
plt.xlabel('Date', fontsize=12)
plt.ylabel('Discharge (m³/s)', fontsize=12)
plt.grid(True, linestyle='--', alpha=0.7)
plt.tight_layout()
plt.xticks(rotation=45)
plt.style.use('seaborn-v0_8-whitegrid')

# Add legend
plt.legend(frameon=True, edgecolor='black')

# Show plot
plt.show()


In [18]:
filtered_pd_zwalm_out[['qmodm3s']].to_csv('qmodm3s_output.csv', index=True)


In [ ]:
print(pd_opt_params.columns)
print(pd_opt_params)

Recalculate the results with the parametersets (as doubts about the values saved during calibration)

In [ ]:
pd.DataFrame(pd_opt_params.iloc[0,:].values.reshape(1,-1), columns = pd_opt_params.columns)

In [ ]:

metrics_names_list = [
    'NSE_cal', 'NSE_val', 'NSE_full',
    'mNSE_cal', 'mNSE_val', 'mNSE_full',
    'FHV_cal', 'FHV_val', 'FHV_full',
    'NSE_lf_cal', 'NSE_lf_val', 'NSE_lf_full'
]
print(metrics_names_list)

In [22]:
n_param_sets = pd_opt_params.shape[0]
pd_perf_recalc = pd.DataFrame(columns=metrics_names_list, 
                              index=range(0, n_param_sets))
pd_list = []

for i in range(n_param_sets):
    pd_temp = pd.DataFrame(pd_opt_params.iloc[i, :].values.reshape(1, -1), columns=pd_opt_params.columns)
    
    pd_zwalm_out = PDM(
        P=p_ep_zwalm['total_precipitation_sum'].values, 
        EP=p_ep_zwalm['potential_evaporation_sum'].values, 
        t=p_ep_zwalm['Timestamp'].values,
        area=area_zwalm_new, deltat=deltat, deltatout=deltat_out,
        parameters=pd_temp
    )
    
    pd_zwalm_out = pd_zwalm_out.set_index('Time')
    
    pd_zwalm_out_p2 = pd_zwalm_out[start_p2:end_p2]  # no warmup!
    pd_zwalm_out_p1 = pd_zwalm_out[start_endofwarmup_p1:end_p1]
    
    kwargs_p1 = {'Qmod': pd_zwalm_out_p1['qmodm3s'].values, 'Qobs': Q_day.loc[start_endofwarmup_p1:end_p1, 'streamflow'].values}
    kwargs_p2 = {'Qmod': pd_zwalm_out_p2['qmodm3s'].values, 'Qobs': Q_day.loc[start_p2:end_p2, 'streamflow'].values}
    kwargs_full = {'Qmod': pd_zwalm_out.loc[start_endofwarmup_p1:end_p2, 'qmodm3s'].values, 'Qobs': Q_day.loc[start_endofwarmup_p1:end_p2, 'streamflow'].values}

    # Calibration metrics
    mnse_cal = mNSE(**kwargs_p1)
    nse_cal = NSE(**kwargs_p1)
    fhv_cal = FHV(**kwargs_p1)
    nse_lf_cal = NSE_LF(**kwargs_p1)

    # Validation metrics
    mnse_val = mNSE(**kwargs_p2)
    nse_val = NSE(**kwargs_p2)
    fhv_val = FHV(**kwargs_p2)
    nse_lf_val = NSE_LF(**kwargs_p2)

    # Full metrics
    mnse_full = mNSE(**kwargs_full)
    nse_full = NSE(**kwargs_full)
    fhv_full = FHV(**kwargs_full)
    nse_lf_full = NSE_LF(**kwargs_full)

    # Insert results
    pd_perf_recalc.iloc[i, :] = [
        nse_cal, nse_val, nse_full,
        mnse_cal, mnse_val, mnse_full,
        fhv_cal, fhv_val, fhv_full,
        nse_lf_cal, nse_lf_val, nse_lf_full
    ]


In [ ]:
print(pd_perf_recalc)

In [ ]:
from matplotlib.ticker import FormatStrFormatter
x = pd_perf_recalc['NSE_val']
y = pd_perf_recalc['NSE_lf_val']

# Identify the point with the highest NSE_val
highest_nse_index = pd_perf_recalc['NSE_val'].astype(float).idxmax()
highest_nse_point = pd_perf_recalc.loc[highest_nse_index]

print(highest_nse_index)
print(highest_nse_point)

# Separate points into green and red based on NSE_LF value
pd_perf_recalc['Color'] = np.where(pd_perf_recalc['NSE_lf_val'] < highest_nse_point['NSE_lf_val'], 'red', 'green')

# Count green and red points (excluding the highest NSE point from green)
green_count = (pd_perf_recalc['Color'] == 'green').sum() - 1
red_count = (pd_perf_recalc['Color'] == 'red').sum()

# Filter out points below y = -1
visible_df = pd_perf_recalc[pd_perf_recalc['NSE_lf_val'] >= -1]
excluded_count = (pd_perf_recalc['NSE_lf_val'] < -1).sum()

# Plot the data
plt.figure(figsize=(6.3, 3.5))
for color, group in visible_df.groupby('Color'):
    label = 'Better $\mathrm{NSE}_{\mathrm{LF}}$' if color == 'green' else 'Worse $\mathrm{NSE}_{\mathrm{LF}}$'
    plt.scatter(group['NSE_val'], group['NSE_lf_val'], label=f"{label}", 
                color=color, alpha=0.7, edgecolor='k', s=50)

# Highlight the point with the highest NSE_val, if it's visible
if highest_nse_point['NSE_lf_val'] >= -1:
    plt.scatter(highest_nse_point['NSE_val'], highest_nse_point['NSE_lf_val'], 
                color='blue', edgecolor='black', s=50, label='$\mathrm{NSE}_{\mathrm{LF}}$ of Highest NSE')

plt.xlabel('NSE in Validation Period', fontsize=12)
plt.ylabel('$\mathrm{NSE}_{\mathrm{LF}}$ in Validation Period', fontsize=12)
plt.ylim(bottom=-1)  # Set Y-axis lower limit
plt.ylim(top=1.1)  # Set Y-axis lower limit
plt.grid(True, linestyle='--', alpha=0.7)
plt.legend(fontsize = 9, frameon=True, edgecolor='black')
plt.tight_layout()
plt.xticks(rotation=45, fontsize = 10)
plt.yticks(fontsize = 10)
plt.style.use('seaborn-v0_8-whitegrid')
plt.axhline(y=1, color='black', linewidth=3)

plt.gca().xaxis.set_major_formatter(FormatStrFormatter('%.1f'))
plt.xticks(np.arange(0, 0.5, 0.1))
plt.savefig("NSE_LF_PDM.png", dpi=1000, bbox_inches='tight') 
plt.show()

# Display the counts
print("Green count:", green_count)
print("Red count:", red_count)
print("Points not plotted (NSE_LF_val < -1):", excluded_count)



In [ ]:
fig, axes = plt.subplots(4,3, constrained_layout = True, figsize = (7,7))
iter = 0
for i in range(4):
    for j in range(3):
        axes[i,j].scatter(x = pd_opt_params.iloc[:,iter], y = pd_perf_recalc['NSE_cal'], edgecolor = 'k')
        axes[i,j].set_xlabel(pd_opt_params.columns[iter])
        iter = iter + 1
fig.suptitle('NSE on calibration set')
fig.supylabel('NSE')

In [ ]:
print(pd_opt_params.iloc[:,0])
print(pd_perf['NSE_val'])

Same Figure but now limit the axes

In [ ]:
fig, axes = plt.subplots(4,3, constrained_layout = True, figsize = (7,7))
iter = 0
for i in range(4):
    for j in range(3):
        axes[i,j].scatter(x = pd_opt_params.iloc[:,iter], y = pd_perf['NSE_cal'], edgecolor = 'k')
        axes[i,j].set_xlabel(pd_opt_params.columns[iter])
        #axes[i,j].set_ylabel('NSE')
        axes[i,j].set_ylim(0, pd_perf['NSE_cal'].max()+0.05)
        iter = iter + 1
fig.suptitle('NSE on calibration set')
fig.supylabel('NSE')

In [28]:
#dict with names in math font
names_dict = {'cmax':'$c_{max}$','cmin':'$c_{min}$','b':'$b$','be':'$b_e$','k1':'$k_1$','k2':'$k_2$','kb':'$k_b$','kg':'$k_g$','St':'$S_t$','bg':'$b_g$','tdly':'$t_d$','qconst':'$Q_c$'}

In [ ]:
pd_opt_params_plot = pd_opt_params.drop('bg',axis = 1)
print(pd_opt_params)
fig, axes = plt.subplots(4,3, constrained_layout = True, figsize = (7,7))
iter = 0
for i in range(4):
    for j in range(3):
        if iter < pd_opt_params_plot.shape[1]:
            axes[i,j].scatter(x = pd_opt_params_plot.iloc[:,iter], y = pd_perf_recalc['NSE_full'], edgecolor = 'k')
            param_name = pd_opt_params_plot.columns[iter]
            axes[i,j].set_xlabel(names_dict[param_name])
            left_bound = bounds_dict[param_name][0]
            right_bound = bounds_dict[param_name][1]
            buffer = 0.05*(right_bound - left_bound)
            axes[i,j].set_xlim(left_bound-buffer, right_bound+buffer)
            axes[i,j].set_ylim(0.5, pd_perf['NSE_cal'].max()+0.05)
            iter = iter + 1
        else:
            fig.delaxes(axes[i,j]) #delete empty subplot!
#fig.suptitle('NSE on full set')
fig.supylabel('NSE')
fig.savefig('Figures/Figures_chapter_PDM/dottyplot_nsefull.pdf',format = 'pdf')

 Count number of parmaetersets with NSE below 0.5

In [ ]:
print(sum(pd_perf_recalc['NSE_full'] < 0.5))
print(sum(pd_perf_recalc['NSE_full'] < 0))

So it seems acceptable to skip the few very bad perforer with NSE below

### Best dataset

In [ ]:
pd_perf_recalc.head()

Idea: determine best dataset based on best NSE and mNSE value for validation set. (paper on mNSE https://agupubs.onlinelibrary.wiley.com/doi/abs/10.1029/1998WR900018). Goal = highest weighted sum. Update: Adapted to just best NSE value

In [ ]:
sorted_NSE_val = pd_perf_recalc.sort_values('NSE_val', ascending=False)
# sorted_NSE_val['NSE_val_score'] = np.arange(1,51)
# sorted_mNSE_val = sorted_NSE_val.sort_values('mNSE_val',ascending=False)
# sorted_mNSE_val['mNSE_val_score'] = np.arange(1,51)
# sorted_NSE_full = sorted_mNSE_val.sort_values('NSE_full',ascending=False)
# sorted_NSE_full['NSE_full_score'] = np.arange(1,51)
# sorted_NSE_full['total_score'] = sorted_NSE_full['mNSE_val_score'] + sorted_NSE_full['NSE_val_score']
# sorted_total_score_val = sorted_NSE_full.sort_values('total_score')
#sorted_total_score_val.head(10)
sorted_NSE_val['score'] = sorted_NSE_val['NSE_val']
sorted_score = sorted_NSE_val.sort_values('score',ascending=False)
sorted_score.head(15)


Plotting the best dataset according to validation data

In [ ]:
index_best_param_set = sorted_score.iloc[0,:].name
print('original index of best parameter set: ' + str(index_best_param_set))
best_param_set = pd_opt_params.iloc[index_best_param_set,:]
display(best_param_set)
pd_zwalm_out_opt = PDM(P = p_ep_zwalm['total_precipitation_sum'].values, 
    EP = p_ep_zwalm['potential_evaporation_sum'].values,
    t = p_ep_zwalm['Timestamp'].values,
    area = area_zwalm_new, deltat = deltat, deltatout = deltat_out ,
    parameters = pd.DataFrame(best_param_set.to_dict(),index=[0]))
pd_zwalm_out_opt = pd_zwalm_out_opt.set_index('Time')

Check the NSE

In [ ]:
nse_full_opt = NSE(
        pd_zwalm_out_opt.loc[start_endofwarmup_p1:end_p2,'qmodm3s'].values,
        Q_day.loc[start_endofwarmup_p1:end_p2,'streamflow'].values
)
print('Caculated NSE on full set: ' + str(nse_full_opt) )
print('NSE as obtained during the calibration exercise :' + str(pd_perf.loc[index_best_param_set,'NSE_full']) )

In [ ]:
pd_perf.head(10)

In [ ]:
hvplot.extension('bokeh')
Q_day['streamflow'].hvplot(alpha = 0.7, label = 'observed', line_width = 1.5) * pd_zwalm_out_initial['qmodm3s'].hvplot(
    alpha = 0.7, frame_width = 900, frame_height = 400,label = 'Modelled: initial',line_dash = 'dashed',line_width = 1.5) * pd_zwalm_out_opt['qmodm3s'].hvplot(
    alpha = 0.7,label = 'Modelled: optimised', line_dash = 'dashed', line_width = 1.5, color = 'green')

Scatter plot

In [ ]:
print(Q_day['streamflow'])
print(pd_zwalm_out_opt['qmodm3s'])

In [ ]:
fig, ax = plt.subplots()
ax.scatter(Q_day['streamflow'],pd_zwalm_out_opt['qmodm3s'], label = 'optimised parameterset',alpha = 0.5)
ax.scatter(Q_day['streamflow'],pd_zwalm_out_initial['qmodm3s'], label = 'initial parameterset',alpha = 0.5)
ax.legend()

QQ-plot

In [ ]:
length = len(pd_zwalm_out_initial)
quantiles = np.linspace(0, 1, length)
# obs_quan = np.quantile(Q_day['Value'].dropna(),quantiles)
# mod_opt_quan = np.quantile(pd_zwalm_out_opt['qmodm3s'],quantiles, method = 'inverted_cdf')
# mod_init_quan = np.quantile(pd_zwalm_out_initial['qmodm3s'],quantiles, method = 'inverted_cdf')

nan_bool = Q_day['streamflow'].isna()
obs_quan = np.sort(Q_day['streamflow'].dropna())
mod_opt_quan = np.sort(pd_zwalm_out_opt['qmodm3s'][~nan_bool])
mod_init_quan = np.sort(pd_zwalm_out_initial['qmodm3s'][~nan_bool])

fig, ax = plt.subplots()
ax.plot(obs_quan, mod_opt_quan,marker = 'o',label = 'Optimised')
ax.plot(obs_quan, mod_init_quan, marker = 'o', label = 'Initial')
ax.plot(obs_quan, obs_quan, label = 'Ideal')
ax.legend()
ax.set_xlabel(r'$Q_{obs}$ [m$^3$/s]')
ax.set_ylabel(r'$Q_{mod}$ [m$^3$/s]')
ax.set_title('QQ-plot')
fig

In [ ]:
len(nan_bool)

## 20 best NM parametersets

Plotting the 5 best according to validation dataset score

In [42]:
flow_dict_test = {}
Cstar_nm_dict = {}
top_nr = 50
for i in range(top_nr):
    index_temp = sorted_score.iloc[i,:].name
    param_set_temp = pd.DataFrame(pd_opt_params.iloc[index_temp,:].to_dict(), index = [0])
    pd_zwalm_out_temp = PDM(P = p_ep_zwalm['total_precipitation_sum'].values, 
            EP = p_ep_zwalm['potential_evaporation_sum'].values, t = p_ep_zwalm['Timestamp'].values,
            area = area_zwalm_new, deltat = deltat, deltatout = deltat_out ,parameters = param_set_temp
    )
    flow_dict_test[index_temp] = pd_zwalm_out_temp['qmodm3s'][pd_zwalm_out.index >= start_p2]
    Cstar_nm_dict[index_temp] = pd_zwalm_out_temp['Cstar'][pd_zwalm_out.index >= start_p2]


In [ ]:
flow_dict = flow_dict_test
print(flow_dict_test)

In [44]:
nan_bool = nan_bool[nan_bool.index >= start_p2]


In [ ]:
print(flow_dict[49].values[~nan_bool])

In [ ]:
obs_quan = np.sort(Q_day['streamflow'][Q_day.index >= start_p2].dropna())
print(len(obs_quan))

Using sort has the same effect as quantile!

In [ ]:
plt.style.use('seaborn-v0_8-whitegrid')  # Set plot style

# Create figure with specified size
fig, ax = plt.subplots(figsize=(6.3, 3.5))

# Plot the ideal line
ax.plot(obs_quan, obs_quan, label='Ideal Prediction')
temp_quantile = np.sort(flow_dict[29].values[~nan_bool])
ax.plot(obs_quan, temp_quantile, linewidth=1, marker='o', markersize=2, label=29)
# Grid and layout
ax.grid(True, linestyle='--', alpha=0.7)
plt.tight_layout()

# Rotate x-axis ticks if needed (can be skipped for QQ plots, but included per your request)
plt.xticks(rotation=45)

# Show the plot
plt.show() 


In [ ]:
plt.style.use('seaborn-v0_8-whitegrid')  # Set plot style

# Create figure with specified size
fig, ax = plt.subplots(figsize=(6.3, 3.5))



# Get the first key for the prediction with the highest NSE
highest_nse_key = list(flow_dict.keys())[0]  # Assuming the first key corresponds to highest NSE

# Plot each parameter set
for key in flow_dict.keys():
    temp_quantile = np.sort(flow_dict[key].values[~nan_bool])
    
    # Apply a different style for the first key (highest NSE)
    if key == highest_nse_key:
        highest_nse_line, = ax.plot(obs_quan, temp_quantile, linewidth=3, color='red', marker='d', markersize=5, label='Prediction with Highest NSE in Validation Period', zorder=2)
    else:
        ax.plot(obs_quan, temp_quantile, linewidth=1, marker='o', markersize=2, label=key, zorder=1)

# Plot the ideal line
ideal_line, = ax.plot(obs_quan, obs_quan, label='Ideal Prediction',linewidth=3, color='black', zorder=3)
# Grid and layout
ax.grid(True, linestyle='--', alpha=0.7)
plt.tight_layout()

# Rotate x-axis ticks if needed (can be skipped for QQ plots, but included per your request)
plt.xticks(rotation=45)

# Add legend with the correct handles
ax.legend(handles=[ideal_line, highest_nse_line], 
          labels=['Ideal Prediction', 'Highest NSE'], 
          frameon=True, edgecolor='black')

# Add axis labels and styling
plt.xlabel('Discharge (m³/s)', fontsize=12)
plt.ylabel('Discharge (m³/s)', fontsize=12)
plt.grid(True, linestyle='--', alpha=0.7)
plt.tight_layout()
plt.xticks(rotation=45)
plt.style.use('seaborn-v0_8-whitegrid')
plt.savefig("50_plots_PDM.png", dpi=1000, bbox_inches='tight') 
# Show the plot
plt.show()



In [ ]:
fig, ax = plt.subplots(figsize = (8,5))
plt.rcParams.update({'font.size': 13})
Q_day['streamflow'].plot(label = 'Geobserveerd', ax = ax) 
pd_zwalm_out_opt['qmodm3s'].plot(label = 'Gekalibreerd PDM', ax =ax)
ax.set_ylabel('$Q$ [m$^3$/s]')
ax.set_xlabel('Tijd')
ax.legend()
pad = Path('Figures/presentation_12_04')
if not os.path.exists(pad):
    os.makedirs(pad)
plt.savefig(pad/'hydrogram.svg')

In [ ]:

# Load the CSV file

file_path = parent / 'data' /'Zwalm_PDM_parameters' / 'NM_NSE_performances_50.csv'
data = pd.read_csv(file_path)

x = data['NSE_val']
y = data['FHV_val']
# Identify the point with the highest NSE_val
highest_nse_index = data['NSE_val'].idxmax()
highest_nse_point = data.loc[highest_nse_index]

print(highest_nse_index)
print(highest_nse_point)

# Separate points into green and red based on FHV proximity to 0
data['Color'] = np.where(np.abs(data['FHV_val']) < np.abs(highest_nse_point['FHV_val']), 'green', 'red')

# Count green and red points
green_count = (data['Color'] == 'green').sum()
red_count = (data['Color'] == 'red').sum() - 1  # Exclude the highest NSE point

# Plot the data
plt.figure(figsize=(6.3, 3.5))
for color, group in data.groupby('Color'):
    label = 'Better FHV' if color == 'green' else 'Worse FHV'
    plt.scatter(group['NSE_val'], group['FHV_val'], label=f"{label}", 
                color=color, alpha=0.7, edgecolor='k', s=50)

# Highlight the point with the highest NSE_val
plt.scatter(highest_nse_point['NSE_val'], highest_nse_point['FHV_val'], 
            color='blue', edgecolor='black', s=50, label='FHV of Highest NSE')

plt.xlabel('NSE in Validation Period', fontsize=12)
plt.ylabel('FHV in Validation Period', fontsize=12)
plt.grid(True, linestyle='--', alpha=0.7)
plt.legend(frameon=True, edgecolor='black')
plt.tight_layout()
plt.xticks(rotation=45)
plt.style.use('seaborn-v0_8-whitegrid')
plt.axhline(y=0, color='black', linewidth=3)
plt.savefig("FHV_PDM_plot.png", dpi=1000, bbox_inches='tight') 
plt.show()

# Display the counts
green_count, red_count









In [51]:
flow_dict_test = {}
Cstar_nm_dict = {}
top_nr = 50
for i in range(top_nr):
    param_set_temp = pd.DataFrame(pd_opt_params.iloc[i,:].to_dict(), index = [0])
    pd_zwalm_out_temp = PDM(P = p_ep_zwalm['total_precipitation_sum'].values, 
            EP = p_ep_zwalm['potential_evaporation_sum'].values, t = p_ep_zwalm['Timestamp'].values,
            area = area_zwalm_new, deltat = deltat, deltatout = deltat_out ,parameters = param_set_temp
    )
    flow_dict_test[i] = pd_zwalm_out_temp['qmodm3s'][pd_zwalm_out.index >= start_p2]
    Cstar_nm_dict[i] = pd_zwalm_out_temp['Cstar'][pd_zwalm_out.index >= start_p2]

In [ ]:
print(flow_dict_test[0].values.mean())

In [ ]:
Q_pred_avg = {key: flow_dict_test[key].values[~nan_bool].mean() for key in flow_dict_test}
print(Q_pred_avg)
Q_pred_avg_df = pd.DataFrame.from_dict(Q_pred_avg, orient='index', columns=['Q_pred_avg'])
Q_pred_avg_df.reset_index(inplace=True)
Q_pred_avg_df.rename(columns={'index': 'key'}, inplace=True)

obs_quan = Q_day['streamflow'][Q_day.index >= start_p2].dropna()
# Normalize by observed mean
obs_quan_avg = obs_quan.mean()
print(obs_quan)
print(obs_quan_avg)
Q_pred_avg_df['Q_pred_avg_2'] = Q_pred_avg_df['Q_pred_avg'] / obs_quan_avg
print(Q_pred_avg_df)

In [ ]:


# Load the CSV file
file_path = parent / 'data' /'Zwalm_PDM_parameters' / 'NM_NSE_performances_50.csv'
data = pd.read_csv(file_path)

# Merge the Q_pred_avg_df with data based on the shared key
merged_df = pd.concat([data.reset_index(drop=True), Q_pred_avg_df['Q_pred_avg_2'].reset_index(drop=True)], axis=1)
print(merged_df)
# Identify the point with the highest NSE_val
highest_nse_index = merged_df['NSE_val'].idxmax()
highest_nse_point = merged_df.loc[highest_nse_index]
print(highest_nse_index)
print(highest_nse_point)
# Define color condition: closer to 1 = better beta
merged_df['Color'] = np.where(np.abs(merged_df['Q_pred_avg_2'] - 1) < np.abs(highest_nse_point['Q_pred_avg_2'])-1, 'green', 'red')
print(np.abs(highest_nse_point['Q_pred_avg_2']))
# Count green and red points
green_count = (merged_df['Color'] == 'green').sum()
red_count = (merged_df['Color'] == 'red').sum()-1

# Plot
plt.figure(figsize=(6.3, 3.5))
for color, group in merged_df.groupby('Color'):
    label = r'Better $\beta_{\mathrm{KGE}}$' if color == 'green' else r'Worse $\beta_{\mathrm{KGE}}$'
    plt.scatter(group['NSE_val'], group['Q_pred_avg_2'], label=label,
                color=color, alpha=0.7, edgecolor='k', s=50)

# Highlight best NSE point
plt.scatter(highest_nse_point['NSE_val'], highest_nse_point['Q_pred_avg_2'],
            color='blue', edgecolor='black', s=50, label='Best NSE')

plt.xlabel('NSE in Validation Period', fontsize=12)
plt.ylabel(r'$\beta_{\mathrm{KGE}}$ in Validation Period', fontsize=12)
plt.grid(True, linestyle='--', alpha=0.7)
plt.legend(frameon=True, edgecolor='black')
plt.tight_layout()
plt.xticks(rotation=45)
plt.style.use('seaborn-v0_8-whitegrid')
plt.axhline(y=1, color='black', linewidth=3) # target beta
plt.savefig("beta-KGE_PDM.png", dpi=1000, bbox_inches='tight') 
plt.show()

# Display the counts
print("Green points (better beta):", green_count)
print("Red points (worse beta):", red_count)





# Save final parameter set

The set with the best performance (based on NSE and mNSE of the validation set) is used

In [ ]:
best_param_set

In [56]:
best_param_NM = pd.DataFrame(best_param_set.values.reshape(1,-1), columns = pd_init_params.columns)
best_param_NM.to_csv("data/Zwalm_PDM_parameters/NM_opt_param.csv", index = False)

In [57]:
flow_dict_test = {}
Cstar_nm_dict = {}
top_nr = 3
for i in range(top_nr):
    index_temp = sorted_score.iloc[i,:].name
    param_set_temp = pd.DataFrame(pd_opt_params.iloc[index_temp,:].to_dict(), index = [0])
    pd_zwalm_out_temp = PDM(P = p_ep_zwalm['total_precipitation_sum'].values, 
            EP = p_ep_zwalm['potential_evaporation_sum'].values, t = p_ep_zwalm['Timestamp'].values,
            area = area_zwalm_new, deltat = deltat, deltatout = deltat_out ,parameters = param_set_temp
    )
    flow_dict_test[index_temp] = pd_zwalm_out_temp['qmodm3s'][pd_zwalm_out.index >= start_p2]
    Cstar_nm_dict[index_temp] = pd_zwalm_out_temp['Cstar'][pd_zwalm_out.index >= start_p2]


In [ ]:
print(flow_dict_test)

In [ ]:
# Step 1: Concatenate all series into a DataFrame
df_ensemble = pd.concat(flow_dict_test.values(), axis=1)


# Step 3: Compute the row-wise mean (ensemble average)
ensemble_mean = df_ensemble.mean(axis=1)

# Step 4: Save to a new DataFrame
df_ensemble_avg = pd.DataFrame({'ensemble_avg_qmodm3s': ensemble_mean})

# Optional: Inspect the result
print(df_ensemble_avg)

In [ ]:
obs_quan = Q_day['streamflow'][Q_day.index >= start_p2]
print(obs_quan)

In [ ]:
obs_quan = obs_quan.to_frame(name='observed')
obs_quan['ensemble_avg_qmodm3s'] = df_ensemble_avg['ensemble_avg_qmodm3s'].values

print(obs_quan)


In [ ]:
NSE(obs_quan['ensemble_avg_qmodm3s'], obs_quan['observed'])

In [ ]:
FHV(obs_quan['ensemble_avg_qmodm3s'], obs_quan['observed'], exceedance_prob=0.02)

In [ ]:
avgobs = obs_quan['observed'].mean(skipna=True)
print(avgobs)
avgsim = obs_quan['ensemble_avg_qmodm3s'].mean()
print(avgsim)
avgsim/avgobs

In [ ]:
NSE_LF(obs_quan['ensemble_avg_qmodm3s'], obs_quan['observed'])

In [ ]:
plt.figure(figsize=(6.3, 3.5))

# Plot observed streamflow
plt.plot(obs_quan.index, obs_quan['observed'], label='Observed Discharge', color='blue', linewidth=1)

# Plot simulated streamflow
plt.plot(obs_quan.index, obs_quan['ensemble_avg_qmodm3s'], label='Simulated Discharge', color='red', linewidth = 1)

# Add axis labels and styling
plt.xlabel('Date', fontsize=12)
plt.ylabel('Discharge (m³/s)', fontsize=12)
plt.grid(True, linestyle='--', alpha=0.7)
plt.tight_layout()
plt.xticks(rotation=45)
plt.style.use('seaborn-v0_8-whitegrid')

# Add legend
plt.legend(frameon=True, edgecolor='black',fontsize = 10)
plt.tick_params(axis='both', which='major', labelsize=10)

plt.savefig("Final_output_PDM.png", dpi=1000, bbox_inches='tight') 
# Show plot
plt.show()